In [ ]:
import numpy as np
from scipy import stats
from scipy import linalg
import pandas as pd
import math

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

sns.set_style()
sns.set_style("ticks")

from __future__ import division

In [ ]:
def set_size(fig):
    fig.set_size_inches(6, 2)
    plt.tight_layout()


## I. System state and measurement simulation

In [ ]:
def trajectories_sim(T=100):
    x = np.zeros(T+1)
    y = np.zeros(T+1)
    x[0] = stats.norm.rvs(0, 1, 1)
    y[0] = float('nan')
    for tt in range(1,T+1):
        x[tt] = 0.7*x[tt-1]+stats.norm.rvs(0, math.sqrt(0.1), 1)
        y[tt] = 0.5*x[tt]+stats.norm.rvs(0, math.sqrt(0.1), 1) 
    y = y[1:T+1]
    return x, y

In [ ]:
T = 2000
x_sim, y_sim = trajectories_sim(T=T)

# Plot state trajectory
fig, ax = plt.subplots()
ax.plot(x_sim)

# Plot measurement trajectory
fig, ax = plt.subplots()
ax.plot(y_sim)

## II. Analytical solution of the filtering problem with the Kalman filter (KF)

In [ ]:
def kalman_filter(y):
    T = len(y)
    
    # System parameters
    A = 0.7
    C = 0.5
    x0 = 0
    P0 = 1
    Q = 0.1
    R = 0.1
    
    # Initialize state and covariance estimates
    x_kf = np.zeros(T+1)
    P_kf = np.zeros(T+1)
    
    x_kf[0] = x0
    P_kf[0] = P0
    
    # Kalman recursions (one-dimensional without matrix multiplication)
    for tt in range(T):
        P_tmin = A*P_kf[tt]*A + Q
        Kt = P_tmin*C/(C*P_tmin*C+R)
        x_kf[tt+1] = A*x_kf[tt]+Kt*(y[tt]-C*A*x_kf[tt])
        P_kf[tt+1] = P_tmin-Kt*C*P_tmin
    return x_kf, P_kf

In [ ]:
x_kf, P_kf = kalman_filter(y_sim)

fig1=plt.figure(1)

plt.subplot(121)
plt.plot(x_sim)
plt.plot(x_kf)
plt.xlim(0,2000)

plt.subplot(122)
plt.plot(x_sim)
plt.plot(x_kf)
plt.xlim(0,100)

# set_size(fig1)
# fig1.savefig('sim_kf.pdf')

## III. Particle filtering solutions

### III.a Resampling strategies
Different resampling strategies are to be considered: 1) multinomial, 2) systematic, and 3) ESS-adaptive resampling. The resampling functions take a set of normalized weights / probabilities and return the ancestor indices from $0,\dots, N - 1$.

In [ ]:
def multinomial_resampling(ws):
    # Determine number of elements
    N = len(ws)
    # Create a sample of uniform random numbers
    u_sample = stats.uniform.rvs(size=N)
    # Transform them appropriately
    u = np.zeros((N,))
    u[N-1] = np.power(u_sample[N-1],1/N)
    for i in range(N-1,0,-1):
        u[i-1] = u[i] * np.power(u_sample[i-1],1/i)
        
    # Output array
    out = np.zeros((N,),dtype=int)
    
    # Find the right ranges
    total = 0.0
    i = 0
    j = 0
    while j < N:
        total += ws[i]
        while j<N and total >u[j]:
            out[j] = i
            j += 1
            
        # Increase weight counter
        i += 1
        
    return out


def systematic_resampling(ws):
    # Determine number of elements
    N = len(ws)        
    # Output array
    out = np.zeros((N,), dtype=int)
    
    # Create one single uniformly distributed number
    u = stats.uniform.rvs()/N
    # Find the right ranges
    total = ws[0]
    j = 0
    for i in range(N):
        while total < u:
            j += 1
            total += ws[j]
            
        # Once the right index is found, save it
        out[i] = j
        u = u+1/N
        
    return out


def stratified_resampling(ws):
    # Determine number of elements
    N = len(ws)        
    # Output array
    out = np.zeros((N,), dtype=int)
    
    # Find the right ranges
    total = ws[0]
    j = 0
    for i in range(N):
        u = (stats.uniform.rvs()+i)/N
        while total < u:
            j += 1
            total += ws[j]
            
        # Once the right index is found, save it
        out[i] = j
        
    return out

### III.b Bootstrap particle filter (BPF)

In [ ]:
A = 0.7
C = 0.5
Q = 0.1
R = 0.1
Q_sqrt = math.sqrt(Q)
R_sqrt = math.sqrt(R)

def trajectories_bpf(y, N=30, sampling='multinomial', ess_max=None):
    # Determine the number of time steps
    T = len(y)
    
    # Save the history
    xs = np.zeros((N, T + 1))
    ancs = np.zeros((N, T + 1), dtype=int)
    ws = np.zeros((N, T + 1))
    
    # Initialisation
    xs[:, 0] = stats.norm.rvs(0, 1, N)
    ws[:, 0] = 1 / N * np.ones((N,))
    ancs[:, 0] = range(N)
    
    # Loop through all time steps
    for t in range(T):
        # Resample
        
        # Calculate effective sample size
        ess = 1 / np.sum(np.power(ws[:, t], 2))
        # If ess_max is not None, only do resampling if ESS is larger
        # than the set threshold
        if ess_max is None or ess < ess_max:
            if sampling == 'multinomial':
                ancs[:, t + 1] = np.random.choice(range(N), size=N, 
                                                  replace=True, p=ws[:, t])
            elif sampling == 'systematic':
                ancs[:, t + 1] = systematic_resampling(ws[:, t])
            else:
                raise ValueError("Sampling scheme {} unknown.".format(sampling))
        else:
            ancs[:, t + 1] = ancs[:, t]
        
        # Propagate
        xs[:, t + 1] = A*xs[ancs[:, t + 1], t]+Q_sqrt*stats.norm.rvs(0, 1, N)
        
        # Weight
        logws = stats.norm.logpdf(y[t], loc=C*xs[:, t + 1],
                               scale=R_sqrt)
        # Substract maximum
        logws -= np.max(logws)
        # Normalize weights to be probabilities
        ws[:, t + 1] = np.exp(logws) / np.sum(np.exp(logws))
        
    return xs, ws, ancs

### III.c Fully adapted particle filter (FAPF)

In [ ]:
A = 0.7
C = 0.5
Q = 0.1
R = 0.1
Q_sqrt = math.sqrt(Q)
R_sqrt = math.sqrt(R)

def trajectories_fapf(y, N=30, sampling='multinomial', ess_max=None):
    # Determine the number of time steps
    T = len(y)
    
    # Save the history
    xs = np.zeros((N, T + 1))
    ancs = np.zeros((N, T + 1), dtype=int)
    ws = 1/N*np.ones((N, T + 1))
    
    # Initialisation
    xs[:, 0] = stats.norm.rvs(0, 1, N)
    #ws[:, 0] = 1 / N * np.ones((N,))
    ancs[:, 0] = range(N)
    
    # Loop through all time steps
    for t in range(T):
        
        # Compute adjustment multipliers
        mu_nu = C*A*xs[:, t]
        Sigma_nu = R+C*Q*C
        nu = stats.norm.pdf(y[t], loc=mu_nu, scale=math.sqrt(Sigma_nu))
        
        # Compute resampling weights
        resampling_weights = nu/np.sum(nu)
                            
        # Calculate effective sample size
        ess = 1 / np.sum(np.power(resampling_weights, 2))
        # If ess_max is not None, only do resampling if ESS is larger
        # than the set threshold
        if ess_max is None or ess < ess_max:
            if sampling == 'multinomial':
                ancs[:, t + 1] = np.random.choice(range(N), size=N, 
                                                  replace=True, p=resampling_weights)
            elif sampling == 'systematic':
                ancs[:, t + 1] = systematic_resampling(resampling_weights)
            else:
                raise ValueError("Sampling scheme {} unknown.".format(sampling))
        else:
            ancs[:, t + 1] = ancs[:, t]
        
        # Propagate
        mu_xx = A*xs[ancs[:,t+1],t]
        K = (Q*C)/(R+C*Q*C)
        mu_xy = mu_xx+K*(y[t]-C*mu_xx)
        Sigma_xy = (1-K*C)*Q
        xs[:,t+1] = mu_xy+stats.norm.rvs(0, 1, N)*math.sqrt(Sigma_xy)
                                            
    return xs, ws, ancs

## IV. Compare filtering solutions

### IV.a Plot BPF, APF and KF 

In [ ]:
N = 100
xs_bpf, ws_bpf, ancs_bpf = trajectories_bpf(y_sim[1:T+1], N=N, sampling='multinomial')
x_bpf = np.sum(xs_bpf*ws_bpf, axis=0)
P_bpf = np.sum(ws_bpf*np.power(xs_bpf-x_bpf,2),axis=0)

In [ ]:
N = 100
xs_fapf, ws_fapf, ancs_fapf = trajectories_fapf(y_sim[1:T+1], N=N, sampling='multinomial')
x_fapf = np.sum(xs_fapf*ws_fapf, axis=0)
P_fapf = np.sum(ws_fapf*np.power(xs_fapf-x_fapf,2),axis=0)

In [ ]:
x_kf, P_kf = kalman_filter(y_sim[1:T+1])

fig, ax = plt.subplots(1,2)
ax[0].plot(x_kf,linewidth=0.5)
ax[0].plot(x_bpf,linewidth=0.5)
ax[0].plot(x_kf-x_bpf,linewidth=0.5)

ax[1].plot(P_kf,linewidth=0.75)
ax[1].plot(P_bpf,linewidth=0.75)
ax[1].plot(P_kf-P_bpf,linewidth=0.75)

set_size(fig)
fig.savefig('kf_bpf.pdf')

fig2, ax2 = plt.subplots(1,2)
ax2[0].plot(x_kf,linewidth=0.5)
ax2[0].plot(x_fapf,linewidth=0.5)
ax2[0].plot(x_kf-x_fapf,linewidth=0.5)

ax2[1].plot(P_kf,linewidth=0.75)
ax2[1].plot(P_fapf,linewidth=0.75)
ax2[1].plot(P_kf-P_fapf,linewidth=0.75)

set_size(fig2)
fig2.savefig('kf_fapf.pdf')

### IV.b Influence of $N$, the number of particles.

Influence of $N$, the number of particles on the average absolute difference between particle filter (PF) and the Kalman filter (KF) solutions, i.e. 
$$
\begin{align}
V_{m} = \frac{1}{T}\sum_{i=1}^{T} \left|x_{KF}-x_{PF}\right|\\
V_{\sigma^{2}} = \frac{1}{T}\sum_{i=1}^{T} \left|\sigma^{2}_{KF}-\sigma^{2}_{PF}\right|,
\end{align}
$$
where
$x$ and $\sigma^{2}$ represent the estimates of the mean and variance of the filtering distribution $p(x_{t}|y_{1:T})$.



In [ ]:
mean_dev_bpf = []
mean_dev_fapf = []
mean_dev_var_bpf = []
mean_dev_var_fapf = []

ns = [10,15,20,25,40,50,75,100,125,150,175,200]

for N in ns:
    print(N)
    dev_bpf = []
    dev_fapf = []
    dev_var_bpf = []
    dev_var_fapf = []
    for i in range(10):
        print(i)
        xs_bpf, ws_bpf, ancs_bpf = trajectories_bpf(y_sim[1:T+1], N=N, sampling='multinomial')
        x_bpf = np.sum(xs_bpf*ws_bpf, axis=0)
        P_bpf = np.sum(ws_bpf*np.power(xs_bpf-x_bpf,2),axis=0)
        
        xs_fapf, ws_fapf, ancs_fapf = trajectories_fapf(y_sim[1:T+1], N=N, sampling='multinomial')
        x_fapf = np.sum(xs_fapf*ws_fapf, axis=0)
        P_fapf = np.sum(ws_fapf*np.power(xs_fapf-x_fapf,2),axis=0)
        
        dev_bpf.append(np.mean(np.abs(x_kf-x_bpf)))
        dev_fapf.append(np.mean(np.abs(x_kf-x_fapf)))
        
        dev_var_bpf.append(np.mean(np.abs(P_kf-P_bpf)))
        dev_var_fapf.append(np.mean(np.abs(P_kf-P_fapf)))

    mean_dev_bpf.append(np.mean(dev_bpf))
    mean_dev_fapf.append(np.mean(dev_fapf))
    mean_dev_var_bpf.append(np.mean(dev_var_bpf))
    mean_dev_var_fapf.append(np.mean(dev_var_fapf))
    

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(ns, mean_dev_bpf, 'o-')
ax[0].plot(ns, mean_dev_fapf, 'o-')
ax[1].plot(ns, mean_dev_var_bpf, 'o-')
ax[1].plot(ns, mean_dev_var_fapf, 'o-')

set_size(fig)
fig.savefig('dev_ifo_N.pdf')

## V. Particle genealogy 


Retrieve all ancestor paths. Both the "active" ones that survive until the end of the time horizon and the ones that die prematurely. Plot particle genealogy. 

Remark: plotting the terminated paths considerably slows down plotting. To speed up the process one should comment out the for loop "for traj in dead_particles:".

In [ ]:
def plot_trajectories(N=20, T=100, sampling='multinomial', ess_max=None):
    xs_fapf, ws_fapf, ancs_fapf = trajectories_fapf(y_sim[1:T+1], N, 
                                    sampling, 
                                    ess_max)
    
    # To plot the trajectories of the dead particles in gray
    # We have to store which ones died and save their trajectories
    dead_particles = []

    # Length of the data
    T = xs_fapf.shape[1] - 1
    tt = tqdm_notebook(range(1, T + 1))
    for t in tt:
        # It is not necessary to find the ancestor lines if no 
        # particles died
        difference = np.setdiff1d(range(N), ancs_fapf[:, t])
        if len(difference) == 0:
            continue
            
        dead_ind = np.unique(difference)
        traj = np.zeros((len(dead_ind), t))
        ancestors = dead_ind
        for s in range(t, 0, -1):
            ancestors = ancs_fapf[ancestors, s - 1]
            traj[:, s - 1] = xs_fapf[ancestors, s - 1]
        dead_particles.append(traj)

    # Retract the actual trajectory of all particles that survived
    # until time T
    traj_survived = np.zeros((N, T + 1))
    traj_survived[:, T] = xs_fapf[:, T]
    ancestors = range(N)
    for t in range(T, 0, -1):
        ancestors = ancs_fapf[ancestors, t - 1]
        traj_survived[:, t - 1] = xs_fapf[ancestors, t - 1]
            
    fig, ax = plt.subplots(figsize=(8, 3))

    for traj in dead_particles:
        for i in range(traj.shape[0]):
            ax.plot(traj[i, :], 'o-', linestyle='-', 
                    color='grey', markersize=0.75, lw=0.25,
                    alpha=0.8, antialiased=True);

    for i in range(N):
        ax.plot(traj_survived[i, :], 'o-', linestyle='-', 
                markeredgecolor='r',
                markerfacecolor='r',
                color='k', markersize=1, lw=0.5,
                alpha=1, antialiased=True);
    
    ax.set_xlabel('Time')
    
    fig2, ax = plt.subplots(figsize=(8, 3))
    
    for i in range(N):
        ax.plot(traj_survived[i, 1:100], 'o-', linestyle='-', 
                markeredgecolor='r',
                markerfacecolor='r',
                color='k', markersize=1, lw=0.5,
                alpha=1, antialiased=True);
    
    ax.set_xlabel('Time')
    
    
    fig.savefig('path_fapf.png')
    fig2.savefig('path_fapf_zoom.png')

### V.a Particle genealogy for different resampling strategies.

Multinomial resampling

In [ ]:
plot_trajectories(N=100,T=100)


Systematic resampling

In [ ]:
plot_trajectories(sampling='systematic', N=100,T=2000)

Adaptive resampling

In [ ]:
plot_trajectories(ess_max=50, N=100,T=200)